<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/%E9%98%85%E8%AF%BB%E7%90%86%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 13.9MB/s 
     |████████████████████████████████| 2.9MB 49.9MB/s 
     |████████████████████████████████| 890kB 50.5MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b70bb0efe3ac64b58ec18ff6e0998acba59ab4885039aa6992ee47c1cdadfd75
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import transformers
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer, HfArgumentParser, SquadDataset
from transformers import SquadDataTrainingArguments as DataTrainingArguments
from transformers import Trainer, TrainingArguments

In [3]:
#!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
!cp dev-v2.0.json train-v2.0.json


--2020-11-16 11:50:05--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.05s   

2020-11-16 11:50:05 (83.5 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



In [4]:
logger = logging.getLogger(__name__)

In [5]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


In [6]:
sys.argv = ["run_squad.py", 
            "--model_type", "bert" , 
            "--model_name_or_path", "bert-base-cased",
            "--do_train", 
            "--do_eval",
            "--data_dir","./",
            "--version_2_with_negative",
            "--learning_rate", "3e-5",
            "--num_train_epochs","4",
            "--max_seq_length","384",
            "--doc_stride","128",
            "--output_dir","./wwm_cased_finetuned_squad/",
            "--per_gpu_eval_batch_size=2",
            "--per_gpu_train_batch_size=2",
            "--save_steps","5000"
            ]

In [7]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    # If we pass only one argument to the script and it's the path to a json file,
    # let's parse it to get our arguments.
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
else:
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

In [8]:
print(data_args)

SquadDataTrainingArguments(model_type='bert', data_dir='./', max_seq_length=384, doc_stride=128, max_query_length=64, max_answer_length=30, overwrite_cache=False, version_2_with_negative=True, null_score_diff_threshold=0.0, n_best_size=20, lang_id=0, threads=1)


In [9]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)


11/16/2020 11:50:05 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/16/2020 11:50:05 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='./wwm_cased_finetuned_squad/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=2, per_gpu_eval_batch_size=2, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=4.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov16_11-50-05_c36a3dcf7c07', logging_first_step=False, logging_steps=500, save_steps=5000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_r

In [10]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)

11/16/2020 11:50:05 - INFO - filelock -   Lock 139634614459696 acquired on /root/.cache/torch/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock


11/16/2020 11:50:05 - INFO - filelock -   Lock 139634614459696 released on /root/.cache/torch/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock
11/16/2020 11:50:06 - INFO - filelock -   Lock 139634614450328 acquired on /root/.cache/torch/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock


11/16/2020 11:50:06 - INFO - filelock -   Lock 139634614450328 released on /root/.cache/torch/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock


11/16/2020 11:50:06 - INFO - filelock -   Lock 139637302453696 acquired on /root/.cache/torch/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


11/16/2020 11:50:12 - INFO - filelock -   Lock 139637302453696 released on /root/.cache/torch/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [11]:
data_args

SquadDataTrainingArguments(model_type='bert', data_dir='./', max_seq_length=384, doc_stride=128, max_query_length=64, max_answer_length=30, overwrite_cache=False, version_2_with_negative=True, null_score_diff_threshold=0.0, n_best_size=20, lang_id=0, threads=1)

In [12]:
# Get datasets
is_language_sensitive = hasattr(model.config, "lang2id")
train_dataset = (
    SquadDataset(
        data_args, tokenizer=tokenizer, is_language_sensitive=is_language_sensitive, cache_dir=model_args.cache_dir
    )
    if training_args.do_train
    else None
)
eval_dataset = (
    SquadDataset(
        data_args,
        tokenizer=tokenizer,
        mode="dev",
        is_language_sensitive=is_language_sensitive,
        cache_dir=model_args.cache_dir,
    )
    if training_args.do_eval
    else None
)

11/16/2020 11:50:16 - INFO - filelock -   Lock 139634599548800 acquired on ./cached_train_BertTokenizer_384_v2.lock
convert squad examples to features:   0%|          | 0/11873 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 11873/11873 [00:00<00:00, 722876.64it/s]
11/16/2020 11:52:22 - INFO - filelock -   Lock 139634599548800 released on ./cached_train_BertTokenizer_384_v2.lock
11/16/2020 11:52:22 - INFO - filelock -   Lock 139634172860344 acquired on ./cached_dev_BertTokenizer_384_v2.lock
convert squad examples to features:   0%|          | 0/11873 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future

In [13]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Training
if training_args.do_train:
    trainer.train(
        model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
    )
    trainer.save_model()
    # For convenience, we also re-save the tokenizer to the same directory,
    # so that you can share your model easily on huggingface.co/models =)
    if trainer.is_world_master():
        tokenizer.save_pretrained(training_args.output_dir)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


Step,Training Loss


KeyboardInterrupt: ignored